# Model Building

In [ ]:

def get_model():
    inp = Input(shape=(1, train_sample.shape[1]))
    x = BatchNormalization()(inp)
    x = LSTM(128, return_sequences=True)(x)
    x = Convolution1D(128, (2), activation='relu', padding="same")(x)
    x = Convolution1D(84, (2), activation='relu', padding="same")(x)
    x = Convolution1D(64, (2), activation='relu', padding="same")(x)
    x = Flatten()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(32, activation="relu")(x)
    ttf = Dense(1, activation='relu', name='regressor')(x)

    model = models.Model(inputs=inp, outputs=ttf)
    opt = optimizers.Nadam(lr=0.005)
    model.compile(optimizer=opt, loss='mae', metrics=['mae'])
    return model


def normalize(X_train, X_valid, X_test, normalize_opt, excluded_feat):
    feats = [f for f in X_train.columns if f not in excluded_feat]
    if normalize_opt is not None:
        if normalize_opt == 'min_max':
            scaler = preprocessing.MinMaxScaler()
        scaler = scaler.fit(X_train[feats])
        X_train[feats] = scaler.transform(X_train[feats])
        X_valid[feats] = scaler.transform(X_valid[feats])
        X_test[feats] = scaler.transform(X_test[feats])
    return X_train, X_valid, X_test


kf = KFold(n_splits=K_FOLD, shuffle=True, random_state=1337)
kf = list(kf.split(np.arange(len(train_sample))))

oof_final = np.zeros(len(train_sample))
sub_final = np.zeros(len(submission))
i = 0
while i < NB_MODELS:
    print('Running Model ', i+1)

    oof = np.zeros(len(train_sample))
    prediction = np.zeros(len(submission))

    for _, (train_index, valid_index) in enumerate(kf):

        train_x = train_sample.iloc[train_index]
        train_y = targets.iloc[train_index]

        valid_x = train_sample.iloc[valid_index]
        valid_y = targets.iloc[valid_index]

        # #apply min max scaler on training, validation data
        train_x, valid_x, test_scaled = normalize(train_x.copy(),
                                                  valid_x.copy(),
                                                  test.copy(), 'min_max', [])

        train_x = train_x.values.reshape(train_x.shape[0], 1, train_x.shape[1])
        valid_x = valid_x.values.reshape(valid_x.shape[0], 1, valid_x.shape[1])
        test_scaled = test_scaled.values.reshape(test_scaled.shape[0],
                                                 1, test_scaled.shape[1])

        model = get_model()
        cb_checkpoint = ModelCheckpoint("model.hdf5",
                                        monitor='val_mae',
                                        save_weights_only=True,
                                        save_best_only=True)

        model.fit(train_x, train_y,
                  epochs=EPOCHS, callbacks=[cb_checkpoint],
                  batch_size=BATCH_SIZE, verbose=0,
                  validation_data=(valid_x, [valid_y]))

        model.load_weights("model.hdf5")
        oof[valid_index] += model.predict(valid_x).ravel()
        prediction += model.predict(test_scaled).ravel()/K_FOLD
        K.clear_session()

    # Obtain the MAE for this run.
    model_score = mse(targets, oof, squared=False)/1e6

    if model_score < 2.77:
        print(f'MAE: {model_score:.2f} averaged')
        oof_final += oof/NB_MODELS
        sub_final += prediction/NB_MODELS
        i += 1
    else:
        print(f'MAE: {model_score:.2f} not averaged')


print(f"\nMAE for NN: {mse(targets, oof_final, squared=False):.0f}")
submission['time_to_eruption'] = sub_final
submission.to_csv(PATH_DATA + 'submission.csv', index=False)